# Statistics - Hypothsis testing

## The problem 

You are a data scientist in [booking.com](https://www.booking.com/). You are in charge on developing models to increase the conversion rates of users. The competition is fierce, but your company is the market leader and you are proud to be a part of the essential data science team which is involved in the core of the buisness.  One day, Your CEO is informing you that a new, start-up company, achieves siginifacntly higher conversion rates, and that the company is contemplating buying it. He ask you to plan an accurate experiment to evalute the lift in conversion rate if the start-up would be bought and integrated in the company.  


Design such plan. Consider:
* What inputs you need or missing
* What is the correct methodoligy to apply to fair test the two systems? 
* What are the needed confidance parameters? 

In the notebook about  <a href="../Lesson 2/Statistics - Confidence Intervals.ipynb"> condifdence intervals </a> we've reached the following condfidance interval of the difference between two independent samples (denoted as sample $x$ and sample $y$):

$$\large CI = (\bar{x}-\bar{y}) \pm{1.96 * \sqrt{\dfrac{\bar{s_{x}^{2}}}{n} + \dfrac{\bar{s_{y}^{2}}}{n}}}$$

<font size=3 color='red'><b>Question: Is the above CI forumla enough to design proper AB test as described above? Try it out first.</b></font>

## Hypotheses testing

<font size=3 color='red'><b>Question: In the probelm above, what is the status quo and what's the innovation? On which side are we? </b></font>

### Single known mean Hypotheses test

* Two sided test, comparing against a known single mean $\mu_{0}$:

$$ \large H_{0}: \mu = \mu_0$$
<br>
$$ \large H_{1}: \mu \neq{\mu_0}$$

* One sided (left), comparing against a known single mean $\mu_{0}$:
$$ \large H_{0}: \mu > \mu_0$$
<br>
$$ \large H_{1}: \mu \leq{\mu_0}$$

<font size=3 color='red'><b>Question: How to we know if a test is left or right sided? </b></font>

In both these cases, the statistics (Z or T, depends on the number of samples) we want to calculate is:

$$ Z = \frac{\bar{x} - \mu_0}{SE}$$

where SE is the standard error, $\frac{s}{\sqrt{n}}$. The p-value will be calculated according to this z-score. The p-value for the one-sided test is half of that of the two sided test. 

### Terminoligy

Usually, we are testing two hypotheses, one against the other. The null hypothesis vs. the alternative hypothesis:

* The null hypothesis - Usually denoted $H_0$ - The status quo, the hypothsis which state the current state of affairs.
* The alternative hypothesis - Usually denoted $H_a$ or $H_1$ - Represent the change or innovation we are considering introducing. 
* p-value - The probability of getting a result that is as extreme or more extreme when the null hypothesis is true


Errors:

* Type 1 error - Rejecting the null hypothesis while in fact, it is true - Also called a false postive error:
  * $\alpha$ - The probabliity that will will falsly reject the null hypothesis - i.e. make a type 1 error - Typical values: 0.01, 0.05, 0.1
  * Significance level - $1-\alpha$ - The probablity of **not** making a type 1 error
  
  
* Type 2 error - Rejecting the alternative hypothesis while in fact, it is true - Also called a False negetive:
  * $\beta$ - The probabliity of falsy accept the alternative hypothesis - i.e. make a type 2 error - Typical values: 0.2, 0.05
  * Power - $1 - \beta$ - The probablity of correctly accepting the alternative hypothesis - i.e. the probability of correctly detecting an effect difference

**Usually, as the innovators, our personal goal would be to reject the null hypothesis and accept the alternative hypothesis - So we could adpot our innovative solution as the new standard**

![alt text](https://cdn.scribbr.com/wp-content/uploads/2021/01/type-i-and-ii-error-2-768x488.png)


### Visualization: The null hypothesis reject area

In [1]:
import sys; sys.path.append('../')
from commons.statistics import calculate_normal_density_distribution
import pandas as pd

In [2]:
from ipywidgets import interact
import plotly_express as px

In [3]:
import scipy.stats as st
test_types = ['Two sided', 'One sided - left', 'One sided - right']
def infer_area(x, critical, test_type):
    return 'Rejected Area' if (test_type =='Two sided' and abs(x) > critical or
                               test_type == 'One sided - right' and x > critical or 
                               test_type == 'One sided - left' and x < -1*critical) else 'Acceptance area'

@interact
def illustrate_rejection_and_acceptance_regions(alpha=[0.2, 0.1, 0.05, 0.01], test_type=test_types):
    data = calculate_normal_density_distribution(0,1, 1000)
    critical = st.norm.ppf(1- (alpha if test_type != 'Two sided' else alpha/2))
    print(f'\nThe critical value for a {1-alpha} {test_type} Test is {critical:.4}')
    data['area'] = data.x.apply(lambda x: infer_area(x, critical, test_type))
    y_highet = data[data.x.apply(abs) < critical].y.min()
    fig = px.area(data.sort_values(by='area'), x='x',y='y', color='area', color_discrete_sequence=["blue", "red"])
    return fig.update_layout(title_text=f"Acceptance (Blue) and Rejection (Red) areas for a {test_type} Hypotheses test (Confidance: {1-alpha})", title_x=0.5)
    

interactive(children=(Dropdown(description='alpha', options=(0.2, 0.1, 0.05, 0.01), value=0.2), Dropdown(descr…

<font size=3 color='red'><b>Exercise: According to wikipedia, the average height of the isreali man is 176.5. We've sampled 50 man with age over 25 in the file isreali_man_100_samples. Conduct an hypothesis test to validate or reject wikipedia's claims in confidance levels of 95% and 99% </b></font>

We are coducting a two sided hypothesis test with signle known mean:

$$ \large H_{0}: \mu = 176.5$$
<br>
$$ \large H_{0}: \mu \neq{176.5}$$

In [4]:
height_df_1 = pd.read_csv('../data/hypothesis_tests/isreali_man_50_samples.csv')
height_df_1.head()

,h
0,185.142460
1,184.390968
2,165.045359
3,173.479874
4,172.906312


In [5]:
height_df_1.h.mean()

178.65902383067183

By CLT, we know that the sample mean is normally (n>30) distributed with Let's calculate the sample mean and standard error

In [6]:
## Open the hypothesis_testings.py file under the src folder to view the code:
from src.hypothesis_testings import calc_z_score_and_p_value_given_samples

In [7]:
z_score, p_value = calc_z_score_and_p_value_given_samples(height_df_1, 176.5, is_one_sided=True)
z_score, p_value

(1.705825053813384, 0.04402032848789611)

The resulted z-score is 1.7. The critical value for a 95% confindance interval is 1.96 for a two sided test, thus, we fail to rejcet the null hypothesis and we accept that the average height is indeed 176.5 cm. 

<font size=3 color='red'><b>Exercise: Could we reject the results above if we conduct a left sided hypotheses test?  </b></font>


$$ \large H_{0}: \mu \geq{176.5}$$
<br>
$$ \large H_{0}: \mu < 176.5$$

Answer - The calculations are the same. But since the Z-score is positive, than the anomaly is on the right while the rejection zone is on the left. Conculsion - do not look only at the p-value!

<font size=3 color='red'><b>Exercise: Could we reject the results above if we conduct a right sided hypotheses test?  </b></font>


$$ \large H_{0}: \mu \leq{176.5}$$
<br>
$$ \large H_{0}: \mu > 176.5$$

In [8]:
z_score, p_value = calc_z_score_and_p_value_given_samples(height_df_1, 176.5, is_one_sided=True)
z_score, p_value

(1.705825053813384, 0.04402032848789611)

Answer: We can reject the null hypothesis in the 95% confidance level if the test was a one sided test

<font size=3 color='red'><b>Question: Should we switch to a one-sided hypothesis test to reject the null hypothesis? </b></font>

<font size=3 color='red'><b>Exercise: We've increased the samples to 1000, in the file isreali_man_1000_samples. Do we still fail to reject the null hypothesis in 95% and 99% confidance level? </b></font>

In [9]:
height_df_1 = pd.read_csv('../data/hypothesis_tests/isreali_man_100_samples.csv')

In [10]:
height_df_1.shape

(100, 1)

In [11]:
z_score, p_value = calc_z_score_and_p_value_given_samples(height_df_1, 176.5, is_one_sided=False)
z_score, p_value

(2.4159077188743225, 0.01569603876048797)

We can now reject the two sided null hypothesis in the 95% confidance level. 

### Comparing two unkonwn means

* Two sided test, comparing two means:

$$ \large H_{0}: \mu_{0} = \mu_1$$
<br>
$$ \large H_{1}: \mu_0 \neq{\mu_1}$$

* One sided (left), comparing two means:


$$ \large H_{0}: \mu_0 \geq{\mu_1}$$
<br>
$$ \large H_{1}: \mu_0 < \mu_1$$

The mean of each sample is normally distributed (by CLT), and difference between them as well:

$$
\Large {\text{diff} \sim \mathcal{N}(\mu_0-\mu_1,\,\dfrac{\sigma_0^{2}}{n} + \dfrac{\sigma_1^{2}}{n})\,}
$$


Since the null hypothesis assumes the difference between the populations is 0 (or bigger), the Z statistic we will use is:

$$
\Large \frac{(\bar{x}-\bar{y}) - 0}{\sqrt{\dfrac{\bar{s_{x}^{2}}}{n} + \dfrac{\bar{s_{y}^{2}}}{n}}}
$$

<font size=3 color='red'><b>Question: What is the literal meaning of the above Z statistic?</b></font>

## Power

Power is the probablity of **correctly** rejecting the null hypothesis. Power, and power analysis, are the most important tools for pre-experimentation design and prevention of p-value hacking.

<font size=3 color='red'><b>Question: What is P-value hacking?</b></font>

P-value hacking is the misuse of post-experiment data analysis to find patterns in data that can be presented as statistically significant, thus dramatically increasing and understating the risk of false positives. Some examples:

* Post experiment removal of samples 
* Post experiment addition of samples
* Post experiment Switching from two sided to one sided test 
* Repeadtly running tests until one present the results of interest.

In [12]:
import ipywidgets as widgets
proprotion_1 = widgets.FloatSlider(
    value=0.5,
    min=0.4,
    max=0.6,
    step=.02,
    description='Coin A:'
)

proprotion_2 = widgets.FloatSlider(
    value=.55,
    min=.4,
    max=.6,
    step=.02,
    description='Coin B:'
)

In [13]:
import math

In [14]:
@interact
def illusrate_power(p_1=proprotion_1, p_2=proprotion_2, samples=[50, 100, 500, 1000], show_power=True):
    from statsmodels.stats.power import TTestIndPower
    from src.hypothesis_testings import calc_power
    analysis = TTestIndPower()
    prop_1_distribution = calculate_normal_density_distribution(p_1,math.sqrt(p_1*(1-p_1)/samples), 1000)
    prop_1_distribution['coin_id'] = 'A'
    prop_2_distribution = calculate_normal_density_distribution(p_2,math.sqrt(p_2*(1-p_2)/samples), 1000)
    prop_2_distribution['coin_id'] = 'B'
    if show_power:
        print(f'The power is: {calc_power(p_1, p_2, samples, 0.05)}')
    data = prop_1_distribution.append(prop_2_distribution).sort_values(by='x')
    fig = px.line(data, x='x',y='y', color='coin_id', range_x=(0.2,0.8))
    return fig.update_layout(title_text=f"Distribution of mean A ({p_1}) and mean B ({p_2})", title_x=0.5)
    

interactive(children=(FloatSlider(value=0.5, description='Coin A:', max=0.6, min=0.4, step=0.02), FloatSlider(…

### Power analysis

The test power is determend four fatcors - The two main ones are:
* The true difference between the means, AKA effect size
* The number of samples

But is also effected by:
* The desired type 1 error level $\alpha$
* The desired type 2 error level $\beta$

Power analysis involves estimating one of these four parameters given values for three other parameters. This is a powerful tool in both the design and in the analysis of experiments that we wish to interpret using statistical hypothesis tests.

<font size=3 color='red'><b>Exercise: Use <a href="https://www.statsmodels.org/stable/generated/statsmodels.stats.power.TTestIndPower.power.html#statsmodels.stats.power.TTestIndPower.power"> statsmodel TTestIndPower class </a> to calculate the power and print the power in each of the illustrations above. Print the power if the show_power flag in on</b></font>

Answer: The code was added to the illustration, See the implementation in the hypothesis_testings.py file under the src folder. In the following you'll see an example of the usage.

In [15]:
from src.hypothesis_testings import calc_power

In [16]:
calc_power?

Signature: calc_power(p_1, p_2, n_samples, alpha=0.05)
Docstring: <no docstring>
File:      ~/dev/Python-ML/Lessons/Lesson 3/src/hypothesis_testings.py
Type:      function


In [17]:
calc_power(p_1 = 0.5, p_2=0.55, n_samples=50, alpha=0.05)

0.07852398768985813

So although coin B has a 10% (0.5 -> 0.55) higher effect size, by using only 50 samples we will **fail to reject the null hypothesis in 92% of the cases**!

## AB tests

An AB test is a broader scope experimint in which hypotheses testing is the main statistical tool, and is consists of two parts:

* Experiment design - In this step we define:
  * The group assingment logic
  * The proportion of traffic/population to be used
  * Sub-populations, if any, on which the test will be run
  * The needed number of samples to reach the proper statistical signifance in both power and confidance level
  * The test duration in days/weeks, as derived from the needed number of samples.
  * Etc.

### Solution

The new company is promising a better conversion rate (proportion). Our goal is to understand the buisness needs and translate them into accurate test. To do so, there are few questions we should ask our CEO:

* What is the buisness impact of each 1% relative improvement in conversions? (Or in other words, how much each relative improvement worth to the company)
* What is the buying price of the new company? what is the minimal conversion rate increase to justify the asked price?
* Is the new company supposed to be better on all traffic or a subsegment of the traffic (for example, they might be working only in the US marketplace)
* Do we want to compare the performances by segment or overall?

Let's assume that the input from the CEO that the minimal increase for purchasing the company is 5% higher from the current state, 7% is a good deal and 10% is a great deal and that the new company works on all marketplaces.


#### Proposed design

A proper AB test in that case:

* Will trigger the test on all traffic (no sub-categories)
* Will run the test on whole number of weeks (1, 2, 3 weeks) to avoid biases of day-of-week 
* Will design a two sided hypothesis test with a null hypothesis stating that the new company's conversion rate is identical to ours 
* Will design the test with enough power to detect a 5% relative effect increase with 95% and 99% probabilites (will offer both numbers). (Note that this means that higher effect sizes will also be decteted in even higher probabilities)
* Will design the test with an alpha of 0.05 and 0.01 - to prevent falsy rejecting the null hypothesis
* **Will supply a table holding all possible durations and sample sizes - And allow the stake holders to make a data-driven decion**

<font size=3 color='red'><b>Exercise: Use the solve_power method in TTestIndPower to prepare a table with all possible sample needed. Assume the current conversion rate is 15%. </b></font>

In [18]:
alphes = [0.05, 0.01]
powers = [0.95, 0.99]

In [19]:
from src.hypothesis_testings import calc_needed_samples

In [20]:
calc_needed_samples(0.15, 0.05, 0.05, 0.95)

58910.314791263045

In [21]:
samples_needed = []
curr_conv_rate = 0.15
for relative_inc in [0.05, 0.07, 0.1]:
    for alpha in alphes:
        for power in powers:
            needed_samples = calc_needed_samples(curr_conv_rate, relative_inc, alpha, power)
            samples_needed.append((100 * curr_conv_rate, 
                                   100 * curr_conv_rate*(1 + relative_inc),
                                   int(100 * relative_inc), 
                                   1-alpha, power, int(needed_samples)))           

In [22]:
results = pd.DataFrame(samples_needed, columns=['Current rate','New rate','Relative difference (%)', 'Confidance Level', 'Power','# Samples'])
results.sort_values(by='# Samples', ascending=False)

,Current rate,New rate,Relative difference (%),Confidance Level,Power,# Samples
3,15.0,15.75,5,0.99,0.99,108943
1,15.0,15.75,5,0.95,0.99,83289
2,15.0,15.75,5,0.99,0.95,80759
0,15.0,15.75,5,0.95,0.95,58910
7,15.0,16.05,7,0.99,0.99,55584
5,15.0,16.05,7,0.95,0.99,42495
6,15.0,16.05,7,0.99,0.95,41204
4,15.0,16.05,7,0.95,0.95,30056
11,15.0,16.50,10,0.99,0.99,27237
9,15.0,16.50,10,0.95,0.99,20823


<font size=3 color='red'><b>Question: What are the trade offs in each row of the results table? </b></font>

### Post experimet analysis

Analysis of the results should only be started after the AB test terimenated (according to the pre-defined duration). Avoid doing pre-termination analysis as those can be misleading. 

<font size=3 color='red'><b>Exercise: The AB test was terminated after 1 week on-air, you've gathered 50,000 samples of each group. The mean of the control group was 0.151 while the mean of the treatment (new company) was 0.169. compute a 99% CI for the difference between the treatment and control conversion rate </b></font>

As the expermint owner, we now need to evaluate two things:
1. What is the p-value, and as such, Which hypothesis is to be rejected and which to be excepted?
2. If the null hypothesis is to be rejected, what is the most likely diff and what is a 95% and 99% confidance interval for the diff between the methods? 

Let's start with question 1:

In [23]:
# We will name the A and B group as control and treatment:
control_m, control_s = 0.151, math.sqrt((1-0.151) * 0.151)
treatment_m, treatment_s = 0.169, math.sqrt((1-0.169) * 0.169)
n_samples = 50000
z_score = (control_m - treatment_s) / math.sqrt((control_s ** 2 + treatment_s ** 2) / n_samples)

In [24]:
from src.hypothesis_testings import calc_p_value_from_z_score
calc_p_value_from_z_score(z_score)

0.0

There is 0% Chance to get such extreme results if the null hypothesis is correct, the null hypothesis is to be rejected with 100% certainty. So we now know that there is a difference in the conversion rates, let's calculate a 99% confidance interval that contains the true difference:

In [25]:
from scipy import stats
estimated_diff = treatment_m - control_m
se = math.sqrt(treatment_s**2 / n_samples + control_s**2 / n_samples)
z_half_alpha = stats.norm.ppf(1 - 0.01/2)
diff_ci = (estimated_diff - z_half_alpha * se, estimated_diff, estimated_diff + z_half_alpha * se)
diff_ci

(0.01202942992151482, 0.018000000000000016, 0.02397057007848521)

The most likely difference between the control and the test is 0.018 difference points, which is 1.8% increase to the current conversion rate. The true difference is in 99% certainty somewhere between 1.2% (0.012) and 2.3% (0.023). We should also calculate the relative difference: 

In [26]:
relative_ci = [bound / control_m for bound in diff_ci]
relative_ci

[0.07966509881797895, 0.11920529801324514, 0.15874549720851133]

The true relative difference is in 99% certainty somewhere between 8% and 15%.

<font size=3 color='red'><b>Question: Given the AB test results, what will you recommend the CEO to do?  </b></font>